In [3]:
import torch
import os
from model_hyperbolic import GPTConfig, GPT

checkpoint_path = os.path.join('out-tinystories', 'ckpt.pt')
# Load the checkpoint
checkpoint = torch.load(checkpoint_path, map_location='cpu')

# Load the model arguments from the checkpoint
model_args = checkpoint['model_args']

# Initialize the model with the loaded config
gpt_config = GPTConfig(**model_args)
model = GPT(gpt_config)

# Load the model state dictionary
state_dict = checkpoint['model']

# Remove unwanted prefix if present
unwanted_prefix = '_orig_mod.'
for k in list(state_dict.keys()):
    if k.startswith(unwanted_prefix):
        state_dict[k[len(unwanted_prefix):]] = state_dict.pop(k)

# Load state dict into the model
model.load_state_dict(state_dict)

number of parameters: 0.61M


<All keys matched successfully>

In [4]:
model

GPT(
  (transformer): ModuleDict(
    (wte): Embedding(50304, 12)
    (wpe): Embedding(512, 12)
    (drop): Dropout(p=0.2, inplace=False)
    (h): ModuleList(
      (0-5): 6 x Block(
        (ln_1): LayerNorm()
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=12, out_features=36, bias=False)
          (c_proj): Linear(in_features=12, out_features=12, bias=False)
          (attn_dropout): Dropout(p=0.2, inplace=False)
          (resid_dropout): Dropout(p=0.2, inplace=False)
        )
        (ln_2): LayerNorm()
        (mlp): MLP(
          (c_fc): Linear(in_features=12, out_features=48, bias=False)
          (gelu): GELU(approximate='none')
          (c_proj): Linear(in_features=48, out_features=12, bias=False)
          (dropout): Dropout(p=0.2, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm()
  )
  (lm_head): Linear(in_features=12, out_features=50304, bias=False)
)

In [8]:
# Iterate through the layers and print the value of the 'c' parameter in hyperbolic mode
for i, block in enumerate(model.transformer.h):
    print(block.attn.mode)
    break
    # if isinstance(block.attn, CausalSelfAttention) and block.attn.mode == 'hyperbolic':
    #     print(f"Layer {i} 'c' parameter value: {block.attn.c.item()}")

original
